In [47]:
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel

import itertools
from collections import Counter
from collections import defaultdict

import json
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import pickle

import pandas as pd
import numpy as np

from sklearn.feature_extraction import text
from sklearn.decomposition import LatentDirichletAllocation

In [61]:
import re, nltk, spacy, string

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

from plotly.offline import plot
import plotly.graph_objects as go
import plotly.express as px


In [59]:

import warnings
warnings.filterwarnings('ignore')

In [2]:
listing_detail_df = pd.read_csv('Downloads/data/listings_detail.csv')

/Users/luyuankong/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = listing_detail_df[['id','description']]

In [24]:
df.dropna(subset = ['description'],inplace=True)

/Users/luyuankong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n','',text)
    text = re.sub('and','',text)
    text = re.sub('the','',text)
    
    return text

round1 = lambda x: clean_text_round1(x)

<>:7: DeprecationWarning: invalid escape sequence \[
<>:9: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \[
<>:9: DeprecationWarning: invalid escape sequence \w
<>:7: DeprecationWarning: invalid escape sequence \[
<>:9: DeprecationWarning: invalid escape sequence \w
<ipython-input-26-7f4f4eb375cd>:7: DeprecationWarning: invalid escape sequence \[
  text = re.sub('\[.*?\]', '', text)
<ipython-input-26-7f4f4eb375cd>:9: DeprecationWarning: invalid escape sequence \w
  text = re.sub('\w*\d\w*', '', text)


In [27]:
data_clean = pd.DataFrame(df.description.apply(round1))

In [28]:
def lemmatize(sentence):
    # Init the Wordnet Lemmatizer
    lemmatizer = WordNetLemmatizer()
    # Tokenize: Split the sentence into words
    word_list = nltk.word_tokenize(sentence)
    # Lemmatize list of words and join
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [29]:
data_clean['description'] = data_clean['description'].apply(lemmatize)

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

# Stop words
stop_wds = ['english','\n','airbnb','new york','new','york','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17']
my_stop_words = text.ENGLISH_STOP_WORDS.union(set(stop_wds))

In [52]:
cv = CountVectorizer(stop_words= my_stop_words,
                    min_df=5,
                     ngram_range = (1, 2))
data_cv = cv.fit_transform(data_clean.description)

In [53]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [54]:
data_vectorized = cv.fit_transform(data_clean['description'])

lda_model = LatentDirichletAllocation(n_components=5, # Number of topics
                                      learning_method='online',
                                      random_state=42,       
                                      n_jobs = -1  # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)


In [55]:
display_topics(lda_model, cv.get_feature_names(), 10)


Topic  0
room, guest, place, private, space, bathroom, home, kitchen, shared, living

Topic  1
apartment, bedroom, floor, ha, space, large, building, kitchen, room, window

Topic  2
bed, bedroom, room, apartment, kitchen, queen, tv, size, bathroom, ha

Topic  3
train, minute, park, block, restaurant, walk, away, apartment, subway, manhattan

Topic  4
minute, amenity, apartment, room, building, ll, kitchen, hotel, city, stay


In [5]:
corpus = df.set_index('id')

In [10]:
def preprocess_text(corpus):
    """Takes a corpus in list format and applies basic preprocessing steps of word tokenization,
     removing of english stop words, lower case and lemmatization."""
    processed_corpus = []
    english_words = set(nltk.corpus.words.words())
    english_stopwords = set(stopwords.words('english'))
    wordnet_lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'[\w|!]+')
    for row in corpus:
        word_tokens = tokenizer.tokenize(row)
        word_tokens_lower = [t.lower() for t in word_tokens]
        word_tokens_lower_english = [t for t in word_tokens_lower if t in english_words or not t.isalpha()]
        word_tokens_no_stops = [t for t in word_tokens_lower_english if not t in english_stopwords]
        word_tokens_no_stops_lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in word_tokens_no_stops]
        processed_corpus.append(word_tokens_no_stops_lemmatized)
    return processed_corpus

In [11]:
def nlp_model_pipeline(processed_corpus):
    """Takes processed corpus and produce dictionary, doc_term_matrix and LDA model"""
    # Creates the term dictionary (every unique term in corpus is assigned an index)
    dictionary = Dictionary(processed_corpus)
    # Convert corpus into Document Term Matrix using dictionary prepared above
    doc_term_matrix = [dictionary.doc2bow(listing) for listing in processed_corpus]    
    return dictionary, doc_term_matrix

In [12]:
def LDA_topic_modelling(doc_term_matrix, dictionary, num_topics=3, passes=2):
    # Create an object for LDA model and train it on Document-Term-Matrix
    LDA = LdaModel
    ldamodel = LDA(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=passes)
    return ldamodel

In [13]:
def add_topics_to_df(ldamodel, doc_term_matrix, df, new_col, num_topics):
    # Convert into Per-document topic probability matrix:
    docTopicProbMat = ldamodel[doc_term_matrix]
    docTopicProbDf = pd.DataFrame(index=df.index, columns=range(0, num_topics))
    for i, doc in enumerate(docTopicProbMat):
        for topic in doc:
            docTopicProbDf.iloc[i, topic[0]] = topic[1]
    docTopicProbDf[new_col] = docTopicProbDf.idxmax(axis=1)
    df_topics = docTopicProbDf[new_col]
    # Merge with df
    df_new = pd.concat([df, df_topics], axis=1)
    return df_new

In [14]:
corpus.head()

,description
id,
2595,"Beautiful, spacious skylit studio in the heart..."
3831,Urban retreat: enjoy 500 s.f. floor in 1899 br...
5099,My large 1 bedroom apartment has a true New Yo...
5121,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...
5178,Please don’t expect the luxury here just a bas...


In [15]:
corpus_description = corpus['description'].astype(str)
processed_corpus_description = preprocess_text(corpus_description)
dictionary_description, doc_term_matrix_description = nlp_model_pipeline(processed_corpus_description)

In [ ]:
# ldamodel_description = LDA_topic_modelling(doc_term_matrix_description, dictionary_description, num_topics=3, passes=10)

In [35]:
ldamodel_description5 = LDA_topic_modelling(doc_term_matrix_description, dictionary_description, num_topics=5, passes=10)

In [36]:
ldamodel_description8 = LDA_topic_modelling(doc_term_matrix_description, dictionary_description, num_topics=8, passes=10)

In [37]:
p3 = pyLDAvis.gensim.prepare(ldamodel_description, doc_term_matrix_description, dictionary_description)

In [38]:
p5 = pyLDAvis.gensim.prepare(ldamodel_description5, doc_term_matrix_description, dictionary_description)

In [39]:
p8 = pyLDAvis.gensim.prepare(ldamodel_description8, doc_term_matrix_description, dictionary_description)

In [40]:
p3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.024249 -0.135189       1        1  36.552887
2     -0.159509  0.081919       2        1  32.348907
0      0.183758  0.053270       3        1  31.098213, topic_info=     Category          Freq          Term         Total  loglift  logprob
121   Default  23165.000000         train  23165.000000  30.0000  30.0000
12    Default  26792.000000           bed  26792.000000  29.0000  29.0000
160   Default  28178.000000          walk  28178.000000  28.0000  28.0000
145   Default  14017.000000        minute  14017.000000  27.0000  27.0000
108   Default  21048.000000          park  21048.000000  26.0000  26.0000
148   Default  12437.000000         queen  12437.000000  25.0000  25.0000
152   Default  12421.000000          size  12421.000000  24.0000  24.0000
149   Default  57271.000000          room  57271.000000  23.0000  23.0000
236   Default  24842.000000          away  24842.000000  22.0000  22.0000
568   Default  10717.000000           min  10717.000000  21.0000  21.0000
62    Default  20134.000000        subway  20134.000000  20.0000  20.0000
156   Default  11769.000000       station  11769.000000  19.0000  19.0000
69    Default  29790.000000             2  29790.000000  18.0000  18.0000
71    Default  15681.000000             5  15681.000000  17.0000  17.0000
132   Default   7652.000000           bus   7652.000000  16.0000  16.0000
143   Default  27445.000000        living  27445.000000  15.0000  15.0000
346   Default  11766.000000       walking  11766.000000  14.0000  14.0000
126   Default   9707.000000            10   9707.000000  13.0000  13.0000
29    Default  19843.000000          full  19843.000000  12.0000  12.0000
0     Default  20308.000000             1  20308.000000  11.0000  11.0000
321   Default  15556.000000         close  15556.000000  10.0000  10.0000
323   Default   9638.000000      distance   9638.000000   9.0000   9.0000
75    Default  24406.000000      bathroom  24406.000000   8.0000   8.0000
355   Default   6990.000000        closet   6990.000000   7.0000   7.0000
36    Default  38523.000000       kitchen  38523.000000   6.0000   6.0000
419   Default   7261.000000            15   7261.000000   5.0000   5.0000
493   Default   5478.000000         table   5478.000000   4.0000   4.0000
1285  Default   5056.000000       airport   5056.000000   3.0000   3.0000
260   Default  17488.000000        street  17488.000000   2.0000   2.0000
72    Default   8091.000000             7   8091.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
721    Topic3   3299.465576        museum   3545.607422   1.0961  -5.7489
903    Topic3   3290.298584       grocery   3561.762939   1.0887  -5.7517
108    Topic3  17331.875000          park  21048.861328   0.9737  -4.0901
62     Topic3  16609.339844        subway  20134.119141   0.9756  -4.1327
341    Topic3   4247.321289          stop   4724.238281   1.0616  -5.4963
323    Topic3   8232.052734      distance   9638.033203   1.0103  -4.8346
346    Topic3   9910.596680       walking  11766.512695   0.9964  -4.6490
71     Topic3  12670.915039             5  15681.894531   0.9548  -4.4033
590    Topic3   3762.178223            st   4152.854492   1.0692  -5.6176
236    Topic3  18701.333984          away  24842.845703   0.8840  -4.0140
72     Topic3   6688.683105             7   8091.001465   0.9777  -5.0422
381    Topic3   5480.569336            20   6497.901855   0.9977  -5.2414
133    Topic3   9118.196289       central  12390.541016   0.8614  -4.7324
321    Topic3  10975.845703         close  15556.973633   0.8192  -4.5469
352    Topic3   8558.625977         block  11572.306641   0.8663  -4.7957
164    Topic3   5495.484863             6   6853.775391   0.9471  -5.2387
260    Topic3  11239.637695        street  17488.914062   0.7259  -4.5232
69     Topic3  15628.406250             2  29790.906250   0.5

In [41]:
p5

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.088456  0.030345       1        1  32.498734
2     -0.097655 -0.069268       2        1  28.677412
0      0.155557 -0.173140       3        1  15.397307
3      0.104920  0.198686       4        1  14.560166
4     -0.251278  0.013377       5        1   8.866385, topic_info=     Category          Freq            Term         Total  loglift  logprob
149   Default  59047.000000            room  59047.000000  30.0000  30.0000
160   Default  29621.000000            walk  29621.000000  29.0000  29.0000
121   Default  24725.000000           train  24725.000000  28.0000  28.0000
145   Default  13485.000000          minute  13485.000000  27.0000  27.0000
12    Default  27392.000000             bed  27392.000000  26.0000  26.0000
712   Default  19243.000000            home  19243.000000  25.0000  25.0000
108   Default  23269.000000            park  23269.000000  24.0000  24.0000
236   Default  26549.000000            away  26549.000000  23.0000  23.0000
62    Default  21488.000000          subway  21488.000000  22.0000  22.0000
146   Default  25184.000000             new  25184.000000  21.0000  21.0000
156   Default  11690.000000         station  11690.000000  20.0000  20.0000
36    Default  39210.000000         kitchen  39210.000000  19.0000  19.0000
568   Default  11186.000000             min  11186.000000  18.0000  18.0000
132   Default   7088.000000             bus   7088.000000  17.0000  17.0000
143   Default  27970.000000          living  27970.000000  16.0000  16.0000
131   Default  20384.000000        building  20384.000000  15.0000  15.0000
133   Default  13306.000000         central  13306.000000  14.0000  14.0000
71    Default  16157.000000               5  16157.000000  13.0000  13.0000
148   Default  12482.000000           queen  12482.000000  12.0000  12.0000
152   Default  12520.000000            size  12520.000000  11.0000  11.0000
394   Default   8783.000000           house   8783.000000  10.0000  10.0000
126   Default   9841.000000              10   9841.000000   9.0000   9.0000
72    Default   7388.000000               7   7388.000000   8.0000   8.0000
162   Default  11854.000000            york  11854.000000   7.0000   7.0000
346   Default  12623.000000         walking  12623.000000   6.0000   6.0000
1285  Default   3995.000000         airport   3995.000000   5.0000   5.0000
75    Default  25091.000000        bathroom  25091.000000   4.0000   4.0000
279   Default  11779.000000           fully  11779.000000   3.0000   3.0000
260   Default  17511.000000          street  17511.000000   2.0000   2.0000
69    Default  30755.000000               2  30755.000000   1.0000   1.0000
...       ...           ...             ...           ...      ...      ...
72     Topic5   3618.768799               7   7388.879883   1.7091  -4.4016
145    Topic5   5671.827148          minute  13485.792969   1.5568  -3.9522
328    Topic5   2121.572021           hotel   3990.445801   1.7912  -4.9355
394    Topic5   3554.604980           house   8783.386719   1.5183  -4.4194
604    Topic5   1144.576172          number   1869.341553   1.9324  -5.5526
897    Topic5   2532.673096              24   5989.854004   1.5621  -4.7584
606    Topic5   2527.499756           phone   6436.978516   1.4881  -4.7605
156    Topic5   3543.571533         station  11690.277344   1.2293  -4.4225
261    Topic5   1955.921387  transportation   4619.813477   1.5634  -5.0168
599    Topic5   1593.212036          hidden   3519.913086   1.6302  -5.2219
372    Topic5   1507.921021          public   3268.123047   1.6494  -5.2770
1725   Topic5    924.485657            self   1429.331787   1.9872  -5.7662
126    Topic5   2622.356934              10   9841.500977   1.1004  -4.7236
160    Topic5   4748.157227            walk  29621.449219   0.5922  -4.1299
617    Topic5   1721.729492              30   4757.786621   1.4064  -5.1444
383    